# Entanglement measure

There are various entanglement measures [wiki-link](https://en.wikipedia.org/wiki/Quantum_entanglement#As_a_measure_of_entanglement) with different meanings:

**Operational entanglement measures**:
1. Distillable entanglement
2. Entanglement cost

**Abstractly defined based on convex roof**:
1. Concurrence
2. Entanglement of formation (EOF)
3. Geometric measure of entanglement (GME)

**Distance measure**:
1. Relative entropy of entanglement (REE)

These measures are zero for all SEP states, positive for ENT states, and non-decreasing under local operations and classical communication (LOCC) [quantiki-link](https://www.quantiki.org/wiki/entanglement-measure). In the following tutorial, we will focus on the bipartite case.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import numqi

## Abstractly defined based on convex roof

### Entanglement of formation

Given a density matrix $\rho$, its von Neumann entropy is defined as

$$ S\left(\rho\right)=-\mathrm{Tr}\left[\rho\ln\rho\right]. $$

For a bipartite system, the entanglement of formation (EOF) of a pure state $\left|\psi\right\rangle \in\mathcal{H}_{A}\otimes\mathcal{H}_{B}$ is defined as

$$ E_f\left(\left|\psi\right\rangle \right)=S\left(\rho_{A}\right)=S\left(\rho_{B}\right) $$

where $\rho_A,\rho_B$ is its reduced density matrix on subsystem $A$ and $B$ respectively

$$ \rho_{A}=\mathrm{Tr}_{B}\left[\left|\psi\right\rangle \left\langle \psi\right|\right],\rho_{B}=\mathrm{Tr}_{A}\left[\left|\psi\right\rangle \left\langle \psi\right|\right]. $$

For a bipartite density matrix $\rho$, its EOF is defined as

$$ E_f\left(\rho\right)=\min_{\left\{ \vec{p},\vec{\psi}\right\} }\sum_{i}p_{i}E_f\left(\left|\psi_{i}\right\rangle \right) $$

$$ s.t.\begin{cases}
\rho=\sum_{i}p_{i}\left|\psi_{i}\right\rangle \left\langle \psi_{i}\right|\\
p_{i}>0\\
\langle\psi_{i}|\psi_{i}\rangle=1
\end{cases} $$

For Werner state, its EOF is given in paper [doi-link](https://doi.org/10.1007/s11704-008-0017-8). Below, we use the variational method to calculate the EOF of Werner state as proposed in paper [doi-link](https://doi.org/10.1103/PhysRevA.64.052304)


In [ ]:
dim = 3
alpha_list = np.linspace(-1, 1, 150)
eof_analytical = numqi.state.get_Werner_eof(dim, alpha_list)

For variational method, we need to perform an optimization for each of target density matrix.

In [ ]:
model = numqi.entangle.EntanglementFormationModel(dim, dim, num_term=2*dim*dim)
eof_variational = []
kwargs = dict(num_repeat=1, print_freq=0, tol=1e-9, print_every_round=0)
for x in tqdm(alpha_list): #about 1 min
    model.set_density_matrix(numqi.state.Werner(dim, x))
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    eof_variational.append(theta_optim.fun)
eof_variational = np.array(eof_variational)

Above, the hyper-parameters, e.g. `num_term=2*dim*dim` and `num_repeat=1`, may not the best choice. You can try to change them to get a better result.

In [ ]:
fig,ax = plt.subplots()
ax.plot(alpha_list, eof_analytical, 'x', label='analytical')
ax.plot(alpha_list, eof_variational, label='variational')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_yscale('log')
ax.set_ylabel('EOF')
ax.set_title(f'Werner({dim})')
fig.tight_layout()

For $\alpha\in[-1,\frac{1}{3}]$, the true EOF (cross in the figure) is zero, so it's not shown in the `yscale='log'` figure. We can see that the variational results (solid line) is almost the same as the true EOF (cross) in the whole range$.

*TASK*: try Isotropic state which also has analytical EOF.

### Concurrence

A closely related quantity is the concurrence, which is defined as

$$ C\left(\left|\psi\right\rangle \right)=\sqrt{2\left(1-\mathrm{Tr}\left[\rho_{A}^{2}\right]\right)} $$

for pure states and

$$ C\left(\rho\right)=\min_{\left\{ \vec{p},\vec{\psi}\right\} }\sum_{i}p_{i}C\left(\left|\psi_{i}\right\rangle \right) $$

$$ s.t.\begin{cases}
\rho=\sum_{i}p_{i}\left|\psi_{i}\right\rangle \left\langle \psi_{i}\right|\\
p_{i}>0\\
\langle\psi_{i}|\psi_{i}\rangle=1
\end{cases} $$

for mixed states.

TODO: add `torch.nn.Module` for this task

### Geometric measure of entanglement 

For a given pure state $\vert \psi \rangle$, the geometric measure of entanglement (GME) is defined as the geometric distance between the given state and *fully product* states

$$
E_G(\vert \psi \rangle)=1-\max_{\vert \varphi_{\text{prod}}\rangle}\vert \langle \varphi_{\text{prod}} \vert \psi \rangle\vert^2.
$$

With the convex roof construction, we can extend this measure to the mixed state case as follows,

$$ E_G\left(\rho\right)=\min_{\left\{ \vec{p},\vec{\psi}\right\} }\sum_{i}p_{i}E_G\left(\left|\psi_{i}\right\rangle \right) $$

$$ s.t.\begin{cases}
\rho=\sum_{i}p_{i}\left|\psi_{i}\right\rangle \left\langle \psi_{i}\right|,\\
p_{i}>0,\\
\langle\psi_{i}|\psi_{i}\rangle=1.
\end{cases} $$

If we want to quantify different levels of entanglement, we can generalize the geometric measure of entanglement to the geometric measure of $r$-bounded rank

$$
E_r(\vert \psi \rangle)=1-\max_{\vert\varphi\rangle \in \sigma_{r-1}}\vert \langle \varphi \vert \psi \rangle\vert^2.
$$

Here, $\sigma_{r-1}$ is the states of rank at most $r-1$ (in the bipartite setting, it means Schmidt rank). Obviously, it can be reduced into the form of GME when $r=2$. And similarly, the definition can be also extended into mixed state case,

$$ E_r\left(\rho\right)=\min_{\left\{ \vec{p},\vec{\psi}\right\} }\sum_{i}p_{i}E_r\left(\left|\psi_{i}\right\rangle \right). $$


### Connection between quantum state and subspace

For the entanglement measures defined by **convex roof**, i.e.,

$$ E\left(\rho\right)=\min_{\left\{ \vec{p},\vec{\psi}\right\} }\sum_{i}p_{i}E\left(\left|\psi_{i}\right\rangle \right). $$

There is a useful relation for all these entanglement measures

$$ E\left(\rho\right) \geq E(\text{supp}(\rho)),$$

where $\text{supp}(\rho)$ is the *support space* of the state $\rho$. The entanglement measure for a quantum subspace $\mathcal{S}$ is defined as follows,

$$
E(\mathcal{S})= \min_{\vert \psi \rangle \in \mathcal{S}} E(\vert \psi \rangle).
$$


## Distance measure

### Relative entropy of entanglement

Relative entropy of entanglement (REE) is defined as the follows

$$ E_R\left( \rho \right) =\min_{\sigma \in \mathrm{SEP}} S\left( \rho ||\sigma \right) $$

where $S$ is the quantum relative entropy (von Neumann entropy) defined as

$$ S\left( \rho ||\sigma \right) =\mathrm{Tr}\left[ \rho \ln \rho -\rho \ln \sigma \right] $$

The relative entropy $S$ is jointly convex in its arguments, and non-negative. It is zero if and only if $\rho=\sigma$. In the following, we will discuss Werner state and Isotropic state which has analytical solution for REE.


**Werner state**:

Werner state is defined as [wiki-link](https://en.wikipedia.org/wiki/Werner_state) over $\mathcal{H}_d\otimes \mathcal{H}_d$

$$ \rho_{W,d}\left(\alpha\right)=\frac{1}{d^{2}-d\alpha}I-\frac{\alpha}{d^{2}-d\alpha}\sum_{ij}\left|ij\right\rangle \left\langle ji\right|,\quad\alpha\in\left[-1,1\right] $$

(You might see different definitions for Werner state, you can check [doc-link](../../get_started/quantum_state) for the conversion among different definitions)

Werner state is separable for $\alpha\in\left[-1,\frac{1}{d}\right]$, and entangled for $\alpha\in\left(\frac{1}{d},1\right]$. The REE of Werner state is given in paper "Entanglement measures under symmetry" [doi-link](https://doi.org/10.1103/PhysRevA.64.062307).

$$ E_R\left(\rho_{W,d}(\alpha)\right)=\begin{cases}
0 & \alpha\in\left[-1,\frac{1}{d}\right]\\
S\left(\rho_{W,d}(\alpha)\lvert\rho_{W,d}\left(\frac{1}{d}\right)\right) & \alpha\in\left[\frac{1}{d},1\right]
\end{cases} $$

Below, we can calculate the REE via various methods including the PPT criterion (lower bound), CHA criterion (upper bound) ([doc-link](../cha)), and compare with the analytical solution.

In [ ]:
dim = 3
alpha_list = np.linspace(0, 1, 100, endpoint=False) # alpha=1 is unstable for matrix logarithm
dm_target_list = [numqi.state.Werner(dim, x) for x in alpha_list]

PPT criterion is to solve the following semidefinite program (SDP). 

$$ \min_{\sigma}\;S\left(\rho_{W,d}\left(\alpha\right)\|\sigma\right) $$

$$ s.t.\;\begin{cases}
\rho\succeq0\\
\mathrm{Tr}\left[\rho\right]=1\\
\rho^{\Gamma}\succeq0
\end{cases} $$

It can be approximately evaluated by semidefinite programming (SDP) according to the paper "Semidefinite approximations of the matrix logarithm" [doi-link](https://doi.org/10.1007%2Fs10208-018-9385-0), [arxiv-link](https://arxiv.org/abs/1705.00812), [github/cvxquad](https://github.com/hfawzi/cvxquad).

In [ ]:
ree_analytical = np.array([numqi.state.get_Werner_ree(dim, x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(dm_target_list, dim, dim) #about 30 seconds

Convex hull approximation (CHA) is to solve the following non-convex optimization problem via gradient descent method. The key idea is to formulize the problem as a non-convex optimization problem, and use gradient descent method to find the optimal solution. The gradient of the objective function is solved by gradient back-propagation method from Pytorch framework. We will dive into more details in the following tutorial ([doc-link](../cha)).

In [ ]:
model = numqi.entangle.AutodiffCHAREE((dim,dim), distance_kind='ree')
ree_cha = [] #about 11 seconds
for dm_target_i in tqdm(dm_target_list):
    model.set_dm_target(dm_target_i)
    tmp0 = numqi.optimize.minimize(model, theta0='uniform', tol=1e-12, num_repeat=1, print_every_round=0).fun
    ree_cha.append(tmp0)
ree_cha = np.array(ree_cha)

In [ ]:
fig, ax = plt.subplots()
ax.plot(alpha_list, ree_analytical, "x", label="analytical")
ax.plot(alpha_list, ree_ppt, label="PPT")
ax.plot(alpha_list, ree_cha, label="CHA")
ax.set_xlim(min(alpha_list), max(alpha_list))
ax.set_ylim(1e-13, 1)
ax.set_yscale('log')
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("REE")
ax.set_title(f'Werner({dim})')
ax.legend()
fig.tight_layout()


Be careful, there is NOT a phase transition in the figure. the sharp shift is only due to the logarithmic scale used in the y-axis. We can see that the upper bound (CHA) and lower bound (PPT) are very close to the analytical solution.

**Isotropic state**:

Let's repeat the same procedure for Isotropic state. Isotropic state is defined as [quantiki-link](https://www.quantiki.org/wiki/isotropic-state) over $\mathcal{H}_d\otimes \mathcal{H}_d$

$$ \rho_{I,d}^{\prime}\left(\alpha\right)=\frac{1-\alpha}{d^{2}}I+\frac{\alpha}{d}\sum_{i}\left|ii\right\rangle \left\langle ii\right|,\quad\alpha\in\left[-\frac{1}{d^{2}-1},1\right] $$

(You might see different definitions for Werner state, you can check [doc-link](../../get_started/quantum_state) for the conversion among different definitions)

Isotropic state is separable for $\alpha\in\left[-\frac{1}{d^2-1},\frac{1}{d+1}\right]$, and entangled for $\alpha\in\left(\frac{1}{d+1},1\right]$. The REE of Werner state is given in paper "Bound on distillable entanglement" [doi-link](https://doi.org/10.1103/PhysRevA.60.179).

$$ E_R\left(\rho_{I,d}(\alpha)\right)=\begin{cases}
0 & \alpha\in\left[-\frac{1}{d^{2}-1},\frac{1}{d+1}\right]\\
S\left(\rho_{I,d}(\alpha)\lvert\rho_{I,d}\left(\frac{1}{d+1}\right)\right) & \alpha\in\left[\frac{1}{d+1},1\right]
\end{cases} $$

In [ ]:
dim = 3
alpha_list = np.linspace(0, 1, 100, endpoint=False) # alpha=1 is unstable for matrix logarithm
dm_target_list = [numqi.state.Isotropic(dim, x) for x in alpha_list]

ree_analytical = np.array([numqi.state.get_Isotropic_ree(dim, x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(dm_target_list, dim, dim) #about 30 seconds

model = numqi.entangle.AutodiffCHAREE((dim, dim), distance_kind='ree')
ree_cha = [] #about 11 seconds
for dm_target_i in tqdm(dm_target_list):
    model.set_dm_target(dm_target_i)
    tmp0 = numqi.optimize.minimize(model, theta0='uniform', tol=1e-12, num_repeat=1, print_every_round=0).fun
    ree_cha.append(tmp0)
ree_cha = np.array(ree_cha)

In [ ]:
fig, ax = plt.subplots()
ax.plot(alpha_list, ree_analytical, "x", label="analytical")
ax.plot(alpha_list, ree_ppt, label="PPT")
ax.plot(alpha_list, ree_cha, label="CHA")
ax.set_xlim(min(alpha_list), max(alpha_list))
ax.set_ylim(1e-13, 1)
ax.set_yscale('log')
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("REE")
ax.legend()
ax.set_title(f"Isotropic({dim})")
fig.tight_layout()